In [2]:
# !pip3 install langchain_community langchain-core

In [1]:
%pip install langchain_community langchain-core


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.7/389.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 49.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain_community.llms import Ollama

In [3]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

'\nNeil Armstrong. He stepped foot on the moon on July 20, 1969 as part of the Apollo 11 mission.'

In [4]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'\nCertainly! Here are the expenses you provided, along with an appropriate category for each one:\n\n* Spotify AB by Adyen - Entertainment\n* Beta Boulders Ams Amsterdam Nld - Sports\n* Taxi Utrecht - Transportation\n* Ministerie van Justitie en Veiligheid - Government\n* Etos AMSTERDAM NLD - Grocery Store\n* Bistro Bar Amsterdam - Food and Drink\n\nI hope this helps! Let me know if you need any further assistance.'

In [6]:
%pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 50.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 31.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Read transaction data

In [7]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [8]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [9]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

### Categorise bank transactions with Llama2

In [10]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [11]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [12]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['', 'Of course! Here are the categories for each of the expenses you provided:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. ISS Catering Services De Meern - Food', '4. Taxi Utrecht - Transportation', '5. Etos AMSTERDAM NLD - Grocery', '6. Bistro Bar Amsterdam - Dining']


,Transaction vs category,Transaction,Category
0,,,None
1,Of course! Here are the categories for each of...,Of course! Here are the categories for each of...,None
2,,,None
3,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
4,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
5,3. ISS Catering Services De Meern - Food,3. ISS Catering Services De Meern,Food
6,4. Taxi Utrecht - Transportation,4. Taxi Utrecht,Transportation
7,5. Etos AMSTERDAM NLD - Grocery,5. Etos AMSTERDAM NLD,Grocery
8,6. Bistro Bar Amsterdam - Dining,6. Bistro Bar Amsterdam,Dining


In [13]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['', 'Sure! Here are the categories I would suggest for each of the expenses you provided:', '', '1. Belastingdienst - Taxes', '2. Tesco Breda - Groceries', '3. Monthly Appartment Rent - Housing', '4. Vishandel Sier Amsterdam - Food Delivery', '5. Selling Paintings - Art Sales', '6. Spotify Ab By Adyen - Music Streaming', '7. Tk Maxx Amsterdam Da - Clothing Store', '8. Consulting - Professional Services', '9. Aidsfonds - Charity Donations', '10. TLS BV Inz Ov-Chipkaart - Transportation', '11. Etos Amsterdam - Convenience Store', '12. Beta Boulders Ams Amsterdam - Fitness', '13. Salary - Personal Expenses', '14. Bouldermuur Bv Amsterdam - Fitness', '15. Birtat Restaurant Amsterdam - Dining', '16. Freelancing - Personal Services', '17. Tikkie - Financial Services', '18. Blogging - Personal Expenses', '19. Taxi Utrecht - Transportation', '20. Apple Services - Tech Services', '21. Amazon Lux - Online Shopping', '22. Classpass* Monthly - Fitness', '23. Audible Uk AdblCo/Pymt Gbr - Entertain

In [14]:
categories_df_all

,Transaction vs category,Transaction,Category
0,,,None
1,Sure! Here are the categories I would suggest ...,Sure! Here are the categories I would suggest ...,None
2,,,None
3,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
4,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
5,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
6,4. Vishandel Sier Amsterdam - Food Delivery,4. Vishandel Sier Amsterdam,Food Delivery
7,5. Selling Paintings - Art Sales,5. Selling Paintings,Art Sales
8,6. Spotify Ab By Adyen - Music Streaming,6. Spotify Ab By Adyen,Music Streaming
9,7. Tk Maxx Amsterdam Da - Clothing Store,7. Tk Maxx Amsterdam Da,Clothing Store


In [50]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [15]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Taxes', 'Groceries', 'Housing', 'Food Delivery',
       'Art Sales', 'Music Streaming', 'Clothing Store',
       'Professional Services', 'Charity Donations', 'Transportation',
       'Convenience Store', 'Fitness', 'Personal Expenses', 'Dining',
       'Personal Services', 'Financial Services', 'Tech Services',
       'Online Shopping', 'Entertainment'], dtype=object)

In [16]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [17]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

/var/folders/vm/3_lyvpks41xg6_kw_0xqs_v80000gn/T/ipykernel_2900/2440992137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
3,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
4,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
5,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
6,4. Vishandel Sier Amsterdam - Food Delivery,4. Vishandel Sier Amsterdam,Food and Drinks
7,5. Selling Paintings - Art Sales,5. Selling Paintings,Art Sales
8,6. Spotify Ab By Adyen - Music Streaming,6. Spotify Ab By Adyen,Music Streaming
9,7. Tk Maxx Amsterdam Da - Clothing Store,7. Tk Maxx Amsterdam Da,Clothing
10,8. Consulting - Professional Services,8. Consulting,Services
11,9. Aidsfonds - Charity Donations,9. Aidsfonds,Charity Donations
12,10. TLS BV Inz Ov-Chipkaart - Transportation,10. TLS BV Inz Ov-Chipkaart,Transportation


In [18]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,NaN,NaN,NaN
8,2023-12-22,Aidsfonds,Expense,10.70,NaN,NaN,NaN
9,2023-12-20,Consulting,Income,2641.93,NaN,NaN,NaN


In [19]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)